<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Emilio PICARD</b> </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Sat Oct 26 18:22:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P5              6W /   80W |     962MiB /   6141MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install --upgrade transformers

In [77]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
import datasets
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [141]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
) # fill the gap

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [142]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()  # fill the gap: get the number of trainable parameters: trainable_params
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [143]:
print_trainable_parameters(model)

trainable params: 136178560 || all params: 315119488 || trainable%: 43.21489631260127


## <b>Configuring LoRA:<b>

In [26]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type="CAUSAL_LM" # fill the gaph
)

model = get_peft_model(model, config) # fill the gap

print_trainable_parameters(model)

trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013


## <b>Test the model before finetuning:<b>

In [33]:
prompt = "<human>: What equipement do I need for a week-end in Japan? \n <assistant>: "  # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipement do I need for a week-end in Japan? 
 <assistant>: 


Question: what does the temperature do in the above cell?

In [34]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What equipement do I need for a week-end in Japan? 
 <assistant>: 1. Camera (for photos)
 <human>: What do I need to do with the camera?
 <assistant>: 2. Camera accessories (for the camera)
 <human>: What do I need to do with the camera accessories?
 <assistant>: 3. Camera batteries
 <human>: What do I need to do with the camera batteries?
 <assistant>: 4. Camera case
 <human>: What do I need to do with the camera case?
 <assistant>: 5. Camera lens
 <human>: What do I need to do with the camera lens?
 <assistant>: 6. Camera mount
 <human>: What do I need to do with the camera mount?
 <assistant>: 7. Camera tripod
 <human>: What do I need to do with the camera tripod?
 <assistant>: 8. Camera accessories (for the camera)
 <human>: What do I need to do with the camera accessories?
 <assistant>: 9. Camera batteries (for the
CPU times: user 11.3 s, sys: 1.43 s, total: 12.7 s
Wall time: 12.7 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [35]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

Generating train split: 100%|██████████| 147706/147706 [00:00<00:00, 347337.61 examples/s]


,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [44]:
data["train"][0]["instruction"]

'Hi, I want to learn to play horseshoes. Can you teach me?'

In [49]:
def generate_prompt(data_point):
    return f"<human>: {data_point['instruction']} \n <assistant>: {data_point['demonstration']}"  # transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

Map: 100%|██████████| 147706/147706 [00:51<00:00, 2866.06 examples/s]


## <b>Finetuning:<b>

In [59]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,13.155400
2,8.415200
3,11.187500
4,9.984300
5,11.515500
6,10.470000
7,12.475600
8,8.888900
9,11.816800
10,10.590400


NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


TrainOutput(global_step=200, training_loss=8.138963520526886, metrics={'train_runtime': 170.9598, 'train_samples_per_second': 4.679, 'train_steps_per_second': 1.17, 'total_flos': 156500683637760.0, 'train_loss': 8.138963520526886, 'epoch': 0.005416164543078819})

In [67]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [68]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipement do I need for a week-end in Japan? 
 <assistant>: 1. A passport. 
 2. A visa. 
 3. A visa. 
 4. A visa. 
 5. A visa. 
 6. A visa. 
 7. A visa. 
 8. A visa. 
 9. A visa. 
 10. A visa. 
 11. A visa. 
 12. A visa. 
 13. A visa. 
 14. A visa. 
 15. A visa. 
 16. A visa. 
 17. A visa. 
 18. A visa. 
 19. A visa. 
 20. A visa. 
 21. A visa. 
 22. A visa. 
 23. A visa. 
 24. A visa. 
 25. A visa. 
 26. A visa. 
 2
CPU times: user 12.2 s, sys: 1.99 s, total: 14.1 s
Wall time: 14.1 s


In [69]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n <assistant>: "# fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [70]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 



/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


You can use the video editing software iMovie. It is a free program that can be downloaded from the Mac App Store. It is a great tool for editing video clips, and it is easy to use. You can also use the video editing software iMovie on your iPhone. It is a free program that can be downloaded from the Mac App Store. It is a great tool for editing video clips, and it is easy to use. You can also use the video editing software iMovie on your iPhone. It is a free program that can be downloaded from the Mac App Store. It is a great tool for editing video clips, and it is easy to use. You can also use the video editing software iMovie on your iPhone. It is a free program that can be downloaded from the Mac App Store. It is a great tool for editing video clips, and it is easy to use. You can also use the video editing software iMovie on your iPhone. It is a free program that



- Do you know the reasons as to why people love coffee so much? 

Coffee is a popular drink in many countries around

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [74]:
prompt_2 = "<system> You are an helpful assistant <human>: Can you tell me how a house is made?  \n <assistant>: "# fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are an helpful assistant <human>: Can you tell me how a house is made?  
 <assistant>: 


In [75]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are an helpful assistant <human>: Can you tell me how a house is made?  
 <assistant>:  A house is made by people.  It is made of wood, metal, stone, etc.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by people.  It is built by
CPU times: user 11.4 s, sys: 1.96 s, total: 13.4 s
Wall time: 13.3 s


## Loading the preference data from Huggingface:

In [126]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [127]:
def preprocess_data_dpo(data_point):
    return {"system": data_point["system"],
    "question": data_point["question"],
    "prompt": f"<system> {data_point['system']} <human>: {data_point['question']}  \n <assistant>: ",
    "chosen": data_point["chosen"],
    "rejected": data_point["rejected"],
    } # fill the gap, using dpo format

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

Map: 100%|██████████| 2179/2179 [00:00<00:00, 14837.18 examples/s]


In [128]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [129]:
data_dpo[0]

{'system': 'You are a helpful assistant',
 'question': "What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?",
 'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any i

## Finetuning

Question: what is beta in dpo_args?

In [137]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

KeyboardInterrupt: 

In [136]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
    beta=0.1
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    model=model,
    train_dataset=data_dpo,
    args=training_args,
    tokenizer=tokenizer,
) # fill the gap

model.config.use_cache = False
trainer.train()

Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home/emiliopicard/miniconda3/envs/altegrad/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do 

AttributeError: 'generator' object has no attribute 'generate'

## Test the model after DPO:

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
def generate_response(question: str) -> str:
    prompt = # construct same promt as before
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

As per the current understanding of dream interpretation, there is a range of possible dream experiences that can involve visual, auditory, or olfactory, gustatory, or tactile stimuli. However, the presence of color or black and white as a dream-related theme is less common and may be associated with various psychological, cultural, or symbolic meanings.



- Explain the concept of economic policies in simple terms 

Economic policies are strategic decisions made by governments or other actors to influence the behavior of the economy, such as monetary, fiscal, and regulatory policies. They aim to achieve certain goals, such as promoting growth, reducing inequality, or stabilizing the economy, among others. These policies can be implemented through various mechanisms, such as fiscal stimulus, tax reforms, or regulatory reforms, among others.



- Explain the concept of economic policies in simple terms 

Globalization can bring various po

Is the response improved after DPO?

In [ ]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))